<a href="https://colab.research.google.com/github/mars-city-dev/olds-theory-open-source-exotic-energy-conduit/blob/main/Akashic_Staging_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3
import os
import json
from datetime import datetime
from flask import Flask, render_template_string, jsonify, request
from waitress import serve

# --- CONFIGURATION ---
DATABASE = 'users.db'
APP_ID = 'akashic-staging-v1'

app = Flask(__name__)

# --- DATABASE LOGIC ---
def get_db_connection():
    conn = sqlite3.connect(DATABASE)
    conn.row_factory = sqlite3.Row
    return conn

def init_db():
    """Initializes the SQLite database with the staging registry table."""
    conn = get_db_connection()
    conn.execute('''
        CREATE TABLE IF NOT EXISTS staging_registry (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT NOT NULL,
            type TEXT NOT NULL,
            status TEXT DEFAULT 'Draft',
            energy TEXT DEFAULT '0%',
            intent TEXT,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    # Seed data if empty
    cursor = conn.execute('SELECT COUNT(*) FROM staging_registry')
    if cursor.fetchone()[0] == 0:
        seed_data = [
            ("The Architect's Intent", "Memory", "Indexed", "98%", "Establishing core foundational logic for the vault."),
            ("Data Legacy Protocol v1", "Document", "Syncing", "45%", "Structural documentation for eternal data persistence."),
            ("Staging Area Conceptualization", "Thought", "Draft", "12%", "Initial spark of the Akashic interface."),
            ("Visual Heritage Asset_09", "Media", "Indexed", "100%", "Visual representation of ancestral lineage.")
        ]
        conn.executemany(
            'INSERT INTO staging_registry (title, type, status, energy, intent) VALUES (?, ?, ?, ?, ?)',
            seed_data
        )

    conn.commit()
    conn.close()

# --- ROUTES ---

@app.route('/')
def index():
    """Renders the main dashboard."""
    conn = get_db_connection()
    items = conn.execute('SELECT * FROM staging_registry ORDER BY created_at DESC').fetchall()
    conn.close()

    # Convert sqlite rows to list of dicts for the template
    registry_items = [dict(ix) for ix in items]

    return render_template_string(HTML_TEMPLATE, items=registry_items)

@app.route('/api/sync', methods=['POST'])
def sync_records():
    """Simulates the Akashic synchronization process."""
    # In a real app, this might update statuses in the DB
    return jsonify({"status": "success", "message": "Universal synchronization complete."})

@app.route('/api/add', methods=['POST'])
def add_record():
    """Adds a new data point to the staging area."""
    data = request.json
    conn = get_db_connection()
    conn.execute(
        'INSERT INTO staging_registry (title, type, status, energy, intent) VALUES (?, ?, ?, ?, ?)',
        (data['title'], data['type'], 'Draft', '5%', data.get('intent', ''))
    )
    conn.commit()
    conn.close()
    return jsonify({"status": "success"})

# --- UI TEMPLATE (Tailwind + Lucide) ---
HTML_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Akashic Staging Area | Data Legacy Vault</title>
    <script src="https://cdn.tailwindcss.com"></script>
    <script src="https://unpkg.com/lucide@latest"></script>
    <style>
        @import url('https://fonts.googleapis.com/css2?family=Inter:wght@300;400;600;800&family=JetBrains+Mono:wght@400;700&display=swap');
        body { font-family: 'Inter', sans-serif; background-color: #050505; color: #cbd5e1; }
        .font-mono { font-family: 'JetBrains Mono', monospace; }
        .akashic-glow { box-shadow: 0 0 50px -12px rgba(99, 102, 241, 0.3); }
        .stardust { background-image: url('https://www.transparenttextures.com/patterns/stardust.png'); opacity: 0.15; }
    </style>
</head>
<body class="selection:bg-indigo-500/30">
    <div class="fixed inset-0 overflow-hidden pointer-events-none z-0">
        <div class="absolute top-[-10%] left-[-10%] w-[40%] h-[40%] bg-indigo-900/10 rounded-full blur-[120px]"></div>
        <div class="absolute bottom-[-10%] right-[-10%] w-[40%] h-[40%] bg-purple-900/10 rounded-full blur-[120px]"></div>
        <div class="absolute inset-0 stardust"></div>
    </div>

    <nav class="relative z-10 border-b border-white/5 bg-black/40 backdrop-blur-xl">
        <div class="max-w-7xl mx-auto px-6 h-20 flex items-center justify-between">
            <div class="flex items-center gap-3">
                <div class="p-2 bg-indigo-500/10 rounded-lg border border-indigo-500/20">
                    <i data-lucide="database" class="w-6 h-6 text-indigo-400"></i>
                </div>
                <div>
                    <h1 class="text-xl font-bold tracking-tight text-white flex items-center gap-2">
                        Data Legacy Vault <span class="text-[10px] font-mono text-indigo-400 bg-indigo-400/10 px-2 py-0.5 rounded uppercase tracking-widest">Akashic</span>
                    </h1>
                    <p class="text-[10px] text-slate-500 font-mono uppercase tracking-[0.2em]">Universal Staging Protocol</p>
                </div>
            </div>
            <button onclick="triggerSync()" id="syncBtn" class="flex items-center gap-2 px-5 py-2.5 rounded-full text-sm font-semibold transition-all bg-white text-black hover:bg-indigo-50 hover:scale-105 active:scale-95">
                <i data-lucide="stars" class="w-4 h-4"></i>
                <span>Sync to Record</span>
            </button>
        </div>
    </nav>

    <main class="relative z-10 max-w-7xl mx-auto px-6 py-12">
        <section class="mb-16">
            <div class="flex flex-col md:flex-row justify-between items-end gap-6 mb-8">
                <div class="max-w-2xl">
                    <div class="inline-flex items-center gap-2 px-3 py-1 rounded-full bg-indigo-500/10 border border-indigo-500/20 text-indigo-400 text-xs font-mono mb-4">
                        <div class="w-1 h-1 rounded-full bg-indigo-400 animate-pulse"></div>
                        ACTIVE STAGING SESSION 0x77AF
                    </div>
                    <h2 class="text-4xl md:text-5xl font-extrabold text-white mb-4 leading-tight tracking-tight">
                        The Staging Area of <br />
                        <span class="text-transparent bg-clip-text bg-gradient-to-r from-indigo-400 via-purple-400 to-pink-400">Universal Memory.</span>
                    </h2>
                    <p class="text-slate-400 text-lg leading-relaxed">
                        Python-driven staging for Data Legacy Vault. Refine, index, and validate your data before it ascends to the permanent record.
                    </p>
                </div>
                <div class="bg-white/5 border border-white/10 p-4 rounded-2xl flex gap-8 backdrop-blur-md">
                    <div class="text-center">
                        <p class="text-[10px] text-slate-500 font-mono mb-1 uppercase">Nodes</p>
                        <p class="text-2xl font-bold text-white">{{ items|length }}</p>
                    </div>
                    <div class="w-px h-10 bg-white/10 my-auto"></div>
                    <div class="text-center">
                        <p class="text-[10px] text-slate-500 font-mono mb-1 uppercase">Integrity</p>
                        <p class="text-2xl font-bold text-indigo-400">99.9%</p>
                    </div>
                </div>
            </div>

            <div class="grid grid-cols-1 md:grid-cols-3 gap-6">
                <div class="bg-gradient-to-br from-indigo-500/10 to-transparent border border-indigo-500/20 p-6 rounded-3xl group transition-all">
                    <i data-lucide="shield" class="w-8 h-8 text-indigo-400 mb-4"></i>
                    <h3 class="text-white font-bold text-xl mb-1">SQLite Core</h3>
                    <p class="text-slate-400 text-sm">Local vault persistence powered by Python 3.x.</p>
                </div>
                <div class="bg-gradient-to-br from-purple-500/10 to-transparent border border-purple-500/20 p-6 rounded-3xl group transition-all">
                    <i data-lucide="cpu" class="w-8 h-8 text-purple-400 mb-4"></i>
                    <h3 class="text-white font-bold text-xl mb-1">Waitress Serve</h3>
                    <p class="text-slate-400 text-sm">Production-ready WSGI orchestration for the staging area.</p>
                </div>
                <div class="bg-gradient-to-br from-blue-500/10 to-transparent border border-blue-500/20 p-6 rounded-3xl group transition-all">
                    <i data-lucide="globe" class="w-8 h-8 text-blue-400 mb-4"></i>
                    <h3 class="text-white font-bold text-xl mb-1">Flask 3.0</h3>
                    <p class="text-slate-400 text-sm">Modern web framework routing your eternal legacy.</p>
                </div>
            </div>
        </section>

        <!-- Registry -->
        <section>
            <div class="flex items-center justify-between mb-8">
                <div class="flex items-center gap-3">
                    <i data-lucide="hexagon" class="w-5 h-5 text-indigo-400"></i>
                    <h3 class="text-2xl font-bold text-white">Registry Queue</h3>
                </div>
                <button onclick="openModal()" class="p-2 bg-indigo-500 text-white rounded-full hover:bg-indigo-400 transition-colors">
                    <i data-lucide="plus" class="w-5 h-5"></i>
                </button>
            </div>

            <div class="overflow-hidden rounded-2xl border border-white/10 bg-black/40 backdrop-blur-md">
                <table class="w-full text-left border-collapse">
                    <thead>
                        <tr class="bg-white/5 text-[10px] uppercase tracking-widest text-slate-400 font-mono">
                            <th class="px-6 py-4">Data Point</th>
                            <th className="px-6 py-4">Classification</th>
                            <th class="px-6 py-4">Status</th>
                            <th class="px-6 py-4">Akashic Energy</th>
                            <th class="px-6 py-4">Added</th>
                        </tr>
                    </thead>
                    <tbody class="divide-y divide-white/5 text-sm">
                        {% for item in items %}
                        <tr class="hover:bg-white/5 transition-colors">
                            <td class="px-6 py-5">
                                <div class="flex items-center gap-3">
                                    <div class="w-2 h-2 rounded-full {% if item.status == 'Indexed' %}bg-emerald-400{% elif item.status == 'Syncing' %}bg-amber-400 animate-pulse{% else %}bg-slate-600{% endif %}"></div>
                                    <span class="text-white font-medium">{{ item.title }}</span>
                                </div>
                            </td>
                            <td class="px-6 py-5">
                                <span class="px-2 py-1 rounded bg-white/5 border border-white/10 text-[10px] text-slate-300">{{ item.type }}</span>
                            </td>
                            <td class="px-6 py-5 text-slate-400">{{ item.status }}</td>
                            <td class="px-6 py-5">
                                <div class="flex items-center gap-2">
                                    <div class="flex-1 h-1.5 w-24 bg-white/5 rounded-full overflow-hidden">
                                        <div class="h-full bg-gradient-to-r from-indigo-500 to-purple-500" style="width: {{ item.energy }}"></div>
                                    </div>
                                    <span class="text-xs font-mono">{{ item.energy }}</span>
                                </div>
                            </td>
                            <td class="px-6 py-5 font-mono text-slate-400 text-xs">{{ item.created_at[:10] }}</td>
                        </tr>
                        {% endfor %}
                    </tbody>
                </table>
            </div>
        </section>

        <!-- Manifestation Area -->
        <section class="mt-12 grid grid-cols-1 lg:grid-cols-2 gap-8">
            <div class="bg-black/60 border border-white/10 rounded-3xl p-8 backdrop-blur-xl">
                <div class="flex items-center gap-2 mb-6">
                    <i data-lucide="sparkles" class="w-5 h-5 text-indigo-400"></i>
                    <h4 class="text-lg font-bold text-white">Staging Intent</h4>
                </div>
                <textarea id="quickIntent" class="w-full h-40 bg-white/5 border border-white/10 rounded-2xl p-4 text-slate-300 text-sm focus:outline-none focus:ring-1 focus:ring-indigo-500/50 resize-none mb-4 font-mono" placeholder="Define the core essence of your current data session..."></textarea>
                <div class="flex justify-between items-center">
                    <p class="text-[10px] text-slate-500 font-mono">PROTOCOL: FLASK_STAGING_v3</p>
                    <button class="px-6 py-2 bg-indigo-500/10 text-indigo-400 border border-indigo-500/20 rounded-full text-xs font-bold hover:bg-indigo-500/20 transition-all">Validate Intent</button>
                </div>
            </div>

            <div class="flex flex-col gap-4">
                <div class="flex-1 bg-gradient-to-br from-indigo-900/20 to-black border border-white/10 rounded-3xl p-6 relative overflow-hidden group">
                   <div class="absolute top-0 right-0 p-8 opacity-10">
                     <i data-lucide="infinity" class="w-32 h-32 text-indigo-500"></i>
                   </div>
                   <h4 class="text-white font-bold mb-2">Legacy Persistence</h4>
                   <p class="text-slate-400 text-sm mb-6 max-w-xs">Control how these SQLite records interface with the eternal MariaDB backbone.</p>
                   <div class="flex gap-3">
                     <button class="px-4 py-2 bg-white/5 rounded-xl text-xs">Eternal</button>
                     <button class="px-4 py-2 bg-indigo-500 text-white rounded-xl text-xs font-bold">Standard</button>
                   </div>
                </div>
                <div class="h-32 bg-white/5 border border-white/10 rounded-3xl flex items-center justify-between px-8">
                   <div class="flex items-center gap-4">
                     <div class="w-10 h-10 rounded-full bg-emerald-500/20 flex items-center justify-center">
                       <i data-lucide="clock" class="w-5 h-5 text-emerald-400"></i>
                     </div>
                     <div>
                       <p class="text-[10px] text-slate-500 font-mono uppercase">Next DB Sync</p>
                       <p class="text-lg font-bold text-white tracking-widest uppercase">Live-Active</p>
                     </div>
                   </div>
                </div>
            </div>
        </section>
    </main>

    <div id="modal" class="fixed inset-0 bg-black/80 backdrop-blur-sm z-50 hidden flex items-center justify-center p-6">
        <div class="bg-[#0a0a0a] border border-white/10 rounded-3xl p-8 max-w-md w-full akashic-glow">
            <h3 class="text-xl font-bold text-white mb-4">Index New Data Point</h3>
            <div class="space-y-4">
                <input id="newTitle" type="text" placeholder="Title" class="w-full bg-white/5 border border-white/10 rounded-xl px-4 py-3 text-sm focus:outline-none focus:border-indigo-500">
                <select id="newType" class="w-full bg-white/5 border border-white/10 rounded-xl px-4 py-3 text-sm focus:outline-none focus:border-indigo-500">
                    <option value="Memory">Memory</option>
                    <option value="Thought">Thought</option>
                    <option value="Document">Document</option>
                    <option value="Media">Media</option>
                </select>
                <div class="flex gap-3 mt-6">
                    <button onclick="closeModal()" class="flex-1 px-4 py-3 bg-white/5 text-white rounded-xl text-sm font-bold">Cancel</button>
                    <button onclick="saveRecord()" class="flex-1 px-4 py-3 bg-indigo-500 text-white rounded-xl text-sm font-bold">Index Point</button>
                </div>
            </div>
        </div>
    </div>

    <script>
        lucide.createIcons();

        function triggerSync() {
            const btn = document.getElementById('syncBtn');
            const original = btn.innerHTML;
            btn.innerHTML = '<i data-lucide="zap" class="w-4 h-4 animate-pulse"></i><span>Syncing...</span>';
            lucide.createIcons();

            fetch('/api/sync', { method: 'POST' })
                .then(r => r.json())
                .then(data => {
                    setTimeout(() => {
                        btn.innerHTML = original;
                        lucide.createIcons();
                    }, 2000);
                });
        }

        function openModal() { document.getElementById('modal').classList.remove('hidden'); }
        function closeModal() { document.getElementById('modal').classList.add('hidden'); }

        function saveRecord() {
            const title = document.getElementById('newTitle').value;
            const type = document.getElementById('newType').value;

            fetch('/api/add', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ title, type, intent: '' })
            }).then(() => window.location.reload());
        }
    </script>
</body>
</html>
'''

# --- EXECUTION ---
if __name__ == '__main__':
    # Initialize the local SQLite environment
    init_db()

    # Environment detection
    is_dev = os.environ.get('FLASK_ENV') == 'development'

    if is_dev:
        print("Starting Akashic Staging in Development Mode...")
        app.run(host='0.0.0.0', port=5000, debug=True)
    else:
        print("Akashic Staging Area is live on port 8080 (Waitress).")
        print("Navigate to: http://localhost:8080")
        serve(app, host='0.0.0.0', port=8080)